In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import pearsonr
from scipy.spatial import distance
from matplotlib import style 
from collections import defaultdict
style.use('default')

# nltk libraries and sources
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

import sys
sys.path.append('../')
import utils

[nltk_data] Downloading package punkt to /Users/myra/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/myra/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/myra/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/myra/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/myra/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/myra/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /Users/myra/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/myra/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
package = ['punkt', 'stopwords', 'wordnet', 'omw-1.4']

print('---- punkt')
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

try:
    nltk.data.find('stopwords')
except LookupError:
    print('---- stopwords')
    nltk.download('stopwords')

try:
    nltk.data.find('wordnet')
except LookupError:
    print('---- wordnet')   
    nltk.download('wordnet')

try:
    nltk.data.find('omw-1.4')
except LookupError:
    print('---- omw-1.4')
    nltk.download('omw-1.4')

---- punkt
---- stopwords
---- wordnet
---- omw-1.4


[nltk_data] Downloading package stopwords to /Users/myra/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/myra/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/myra/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [24]:
data_root_folder = "../data/"
stopwords_en = set(stopwords.words('english'))

## Read data

In [25]:
data_train, data_dev = utils.load_data(data_root_folder)
articles = utils.load_articles()

In [31]:
data_train, data_dev = utils.load_data(data_root_folder)
articles = utils.load_articles()
# lower textual data
data_train['essay'] = data_train['essay'].apply(lambda x: x.lower())
articles['text'] = articles['text'].apply(lambda x: x.lower())

# tokenized version of the text
data_train['essay_tok'] = data_train['essay'].apply(lambda x: nltk.word_tokenize(x))
articles['text_tok'] = articles['text'].apply(lambda x: nltk.word_tokenize(x))

# lemmatize
lemmatizer = WordNetLemmatizer()  # lemmatize
stemmer = PorterStemmer()
data_train['essay_lem'] = data_train['essay_tok'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])
articles['text_lem'] = articles['text_tok'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

In [32]:
# source: https://www.geeksforgeeks.org/python-lemmatization-with-nltk/

 
lemmatizer = WordNetLemmatizer()
 
print("rocks :", lemmatizer.lemmatize("rocks"))
print("corpora :", lemmatizer.lemmatize("corpora"))
 
# a denotes adjective in "pos"
print("better :", lemmatizer.lemmatize("better", pos ="a"))

rocks : rock
corpora : corpus
better : good


In [33]:
tok = ' '.join(articles[:1]['text_tok'].item()[:100])
lems = ' '.join(articles[:1]['text_lem'].item()[:100])

print(tok)
print()
print(lems)


'abhorrent ' bottle attack on young rangers fan before celtic game — police have described a bottle attack on an 11-year-old rangers fan before sunday 's league cup semi-final game with celtic as `` abhorrent '' . kraig mackay suffered a large cut to his head after being hit by the bottle while walking in aitkenhead road , glasgow , at about 14:00 on sunday . pictures on social media showed kraig with a large gash and a bandage on his head . police have appealed for help to find the person responsible . the boy was walking with his

'abhorrent ' bottle attack on young ranger fan before celtic game — police have described a bottle attack on an 11-year-old ranger fan before sunday 's league cup semi-final game with celtic a `` abhorrent '' . kraig mackay suffered a large cut to his head after being hit by the bottle while walking in aitkenhead road , glasgow , at about 14:00 on sunday . picture on social medium showed kraig with a large gash and a bandage on his head . police have appeal

# Test on history for early stopping

In [33]:
# to try code on single entry in beginning of training
h_single = pd.read_csv('../output/history_multiinput_empathy.csv', index_col=0)[1:]

In [63]:
h = pd.read_csv('../output/history_multiinput_empathy.csv', index_col=0)
h2 = pd.DataFrame({'epoch': [2,3,4,5,6,7,8,9,10], 'avrg_dev_loss': [0.3, 0.4, 0.3, 0.6, 0.61, 0.2, 0.5, 0.5, 0.5], 'avrg_dev_r2': [-2.71, -2.71, -2.71, -2.71, -2.71, -2.71, -2.71, -2.71, -2.71], 'dev_corr':[0.3, 0.21, 0.2, 0.3, 0.21, 0.2, 0.3, 0.21, 0.2], 'train_corr': [0.5, 0.6, 0.7, 0.5, 0.6, 0.7, 0.5, 0.6, 0.7]}, index=[2,3,4,5,6,7,8,9,10])
h2
h = pd.concat([h, h2])
h

,epoch,avrg_dev_loss,avrg_dev_r2,dev_corr,train_corr
0,0.0,0.316782,-2.719167,-0.797076,-0.475792
1,1.0,0.353231,-26.486209,-0.793982,-0.485336
2,2.0,0.300000,-2.710000,0.300000,0.500000
3,3.0,0.400000,-2.710000,0.210000,0.600000
4,4.0,0.300000,-2.710000,0.200000,0.700000
5,5.0,0.600000,-2.710000,0.300000,0.500000
6,6.0,0.610000,-2.710000,0.210000,0.600000
7,7.0,0.200000,-2.710000,0.200000,0.700000
8,8.0,0.500000,-2.710000,0.300000,0.500000
9,9.0,0.500000,-2.710000,0.210000,0.600000


In [70]:
hist = h_single
early_stop_toleance = 2
hi = h_single
hi = h
worse_loss = 0
for i in range(hi.shape[0]):
    hist = hi[:(i+1)]
    

    all_dev_loss = hist['avrg_dev_loss'].to_numpy()
    if all_dev_loss.shape[0] > 1:  # not possible in first epoch
        if all_dev_loss[-2] <= all_dev_loss[-1]:
            worse_loss += 1
        else:
            worse_loss = 0
    print(worse_loss)
    if worse_loss == early_stop_toleance:
        print('early stopping at epoch', i)
        model_saved_at = np.zeros(hi.shape[0])  # array with 0 and 1, 1 is the one where we have early stopping. This model is saved
        epoch_i = i  # to remove
        model_saved_at[int(epoch_i)] = 1
        hi['early_stopping'] = model_saved_at
        print(model_saved_at)
        print(hi)
        break


0
1
0
1
0
1
2
early stopping at epoch 6
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
    epoch  avrg_dev_loss  avrg_dev_r2  dev_corr  train_corr  early_stopping
0     0.0       0.316782    -2.719167 -0.797076   -0.475792             0.0
1     1.0       0.353231   -26.486209 -0.793982   -0.485336             0.0
2     2.0       0.300000    -2.710000  0.300000    0.500000             0.0
3     3.0       0.400000    -2.710000  0.210000    0.600000             0.0
4     4.0       0.300000    -2.710000  0.200000    0.700000             0.0
5     5.0       0.600000    -2.710000  0.300000    0.500000             0.0
6     6.0       0.610000    -2.710000  0.210000    0.600000             1.0
7     7.0       0.200000    -2.710000  0.200000    0.700000             0.0
8     8.0       0.500000    -2.710000  0.300000    0.500000             0.0
9     9.0       0.500000    -2.710000  0.210000    0.600000             0.0
10   10.0       0.500000    -2.710000  0.200000    0.700000             0.0


In [71]:
h

,epoch,avrg_dev_loss,avrg_dev_r2,dev_corr,train_corr,early_stopping
0,0.0,0.316782,-2.719167,-0.797076,-0.475792,0.0
1,1.0,0.353231,-26.486209,-0.793982,-0.485336,0.0
2,2.0,0.300000,-2.710000,0.300000,0.500000,0.0
3,3.0,0.400000,-2.710000,0.210000,0.600000,0.0
4,4.0,0.300000,-2.710000,0.200000,0.700000,0.0
5,5.0,0.600000,-2.710000,0.300000,0.500000,0.0
6,6.0,0.610000,-2.710000,0.210000,0.600000,1.0
7,7.0,0.200000,-2.710000,0.200000,0.700000,0.0
8,8.0,0.500000,-2.710000,0.300000,0.500000,0.0
9,9.0,0.500000,-2.710000,0.210000,0.600000,0.0
